In [2]:
# importando bibliotecas
library(jsonlite)
library(triangle)
library(httr)
library(MASS)

numero_ativos = 4  # sincronizar com o numero de ativos na carteira pois este valor sera utilizado na hora de montar a media da correlacao
numero_cenarios = 20000 # para o Monte Carlo

# montando a carteira. Depois vamos pegar de algum lugar ou montar uma carteira mais real
carteiraJSON <- '[
{"symbol" : "ALPA4", "Position" : 140900, "Price" : 12.22},
{"symbol" : "ALSC3", "Position" : 49538,  "Price" : 14.88},
{"symbol" : "COCE5", "Position" : 108316, "Price" : 42.70},
{"symbol" : "ITSA4", "Position" : 534292, "Price" : 9.49}
]'


carteira <- fromJSON(carteiraJSON)

#pegando historico de variacao dos precos
query <- "http://api.informativos.io/prices_matrix?start_date=2016-08-24&end_date=2018-08-24&symbols[]=ALPA4&symbols[]=ALSC3&symbols[]=COCE5&symbols[]=ITSA4"
response <- content(GET(url=query), "text")

# transformando num dataframe
data <- as.data.frame(fromJSON(response, simplifyDataFrame = TRUE), stringsAsFactors=FALSE)

#colocando a primeira linha como cabecalho
normalize_data <- function(df) {
  names(df) <- as.character(unlist(df[1,]))
  df[-1,]
}
precos <- normalize_data(data)


# **************** Inicio da Diferenca de Monte Carlo *************************

# fazendo o typecast de char para numeric para poder tratar os valores
precos_numericos <-precos
precos_numericos <- transform(precos,ALPA4 = as.numeric(ALPA4), ALSC3 = as.numeric(ALSC3), COCE5 = as.numeric(COCE5), ITSA4 = as.numeric(ITSA4))
precos_numericos[1] <- NULL

# testando o tipo de precos_numericos para ver se de fato esta como numerico
sapply(precos_numericos,class)

#  ***************** INICIO - Gerando variaveis correlacionadas *************************
# Primeiro passo do algoritmo de Geracao de amostras normais correlacionadas em R
matriz_correlacao <- cor(precos_numericos, use="complete.obs", method = c("pearson"))

media = rep(0,numero_ativos)
x = mvrnorm(numero_cenarios,media,matriz_correlacao)

# Segundo passo do algoritmo
u <- pnorm(x)

# Terceiro passo do algoritmo - Gerando de fato as variaveis correlacionadas
ativo_alpa4 <-qnorm(u[,1], mean(precos_numericos[["ALPA4"]]), sd(precos_numericos[["ALPA4"]]))
ativo_alsc3 <-qnorm(u[,2], mean(precos_numericos[["ALSC3"]]), sd(precos_numericos[["ALSC3"]]))
ativo_coce5 <-qnorm(u[,3], mean(precos_numericos[["COCE5"]]), sd(precos_numericos[["COCE5"]]))
ativo_itsa4 <-qnorm(u[,4], mean(precos_numericos[["ITSA4"]]), sd(precos_numericos[["ITSA4"]]))

# Gerando um dataframe com as variaveis correlacionadas

precos_correlacionados <- data.frame("Date" = seq(1, numero_cenarios, by=1), "ALPA4" = ativo_alpa4, "ALSC3" = ativo_alsc3, "COCE5" = ativo_coce5, "ITSA4" = ativo_itsa4)

#  ***************** FIM - Termina Geracao de variaveis correlacionadas ******************* 

# **************** FIM da Diferenca de Monte Carlo *************************

#criando estrutura que ira guardar os dados aplicados
precos_aplicados <- data.frame("Data" = c(), "Ativo" = c(), "Financeiro" = c(), stringsAsFactors = FALSE)

# iterando sobre a carteira
for (linha_carteira in 1:nrow(carteira)) {
  ativo <- carteira[linha_carteira, "symbol"]
  preco_original <- carteira[linha_carteira, "Price"]
  posicao_original <- carteira[linha_carteira, "Position"]
  financeiro_original <- preco_original * posicao_original
  #iterando sobre os precos_correlacionados
  for (linha_preco in 1:nrow(precos)) {
    # Aplicando na carteira
    change <- sapply(precos_correlacionados[linha_preco, ativo], as.numeric)
    perda_e_ganho <- financeiro_original * change        
    precos_aplicados <- rbind(precos_aplicados, data.frame("Data"=precos_correlacionados[linha_preco, "Date"], "Ativo"=ativo, "Financeiro"=perda_e_ganho))
  }    
}

# ajustando indices do dataframe
rownames(precos_aplicados) <- 1:nrow(precos_aplicados)

#agrupando por dia e somando os retornos
perda_ganho_agrupado <- aggregate(Financeiro ~ Data,precos_aplicados,sum);

print(perda_ganho_agrupado)

#ordenando os dados
ordenado <- perda_ganho_agrupado[order(perda_ganho_agrupado$Financeiro),]

print(ordenado)

#calculando o Var de 95% e 99%
var <- quantile(ordenado$Financeiro, probs=c(0.05, 0.01)) 

#imprimindo resultado
print(var)

ALPA4     COCE5 
"numeric" "numeric"

           ALPA4      COCE5
ALPA4  1.0000000 -0.1236764
COCE5 -0.1236764  1.0000000
       5%        1% 
-261940.6 -284008.7 
